Credit to 
- https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-download-auto-examples-tutorials-run-fasttext-py
- Facebook for fastText

This uses gensim fastText n-gram character embeddings to figure out similarity between resumes and jobs using kaggle datasets I found



In [1]:
import operator
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline

In [3]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Here, we'll learn to work with fastText library for training word-embedding
models, saving & loading them and performing similarity operations & vector
lookups analogous to Word2Vec.



When to use FastText?
---------------------

The main principle behind `fastText <https://github.com/facebookresearch/fastText>`_ 
is that it treats each word as the aggregation of its subwords. Subwords are character n-grams of the word. The vector for a word is simply taken to be the sum of all vectors of its component char-ngrams.


According to a detailed comparison of Word2Vec and FastText in `this notebook <https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Word2Vec_FastText_Comparison.ipynb>`__, fastText does significantly better on syntactic tasks as compared to the original Word2Vec, especially when the size of the training corpus is small. Word2Vec slightly outperforms FastText on semantic tasks though. The differences grow smaller as the size of training corpus increases.


Training time for fastText is significantly higher than the Gensim version of Word2Vec (\ ``15min 42s`` vs ``6min 42s`` on text8, 17 mil tokens, 5 epochs, and a vector size of 100).


fastText can be used to obtain vectors for out-of-vocabulary (OOV) words, by summing up vectors for its component char-ngrams, provided at least one of the char-ngrams was present in the training data.




## Load Jobs Data

In [6]:
# jobs only
# gives df with list of strings (tokenized) as well as lemmatized list of strings
local_jobs_cleaned_lemmatized_tokenized_path = '/Users/richardkuzma/coding/NLP_projects/job_recommender_project/data/cleaned_lemmatized_tokenized_jobs_dataset_madhab.csv'

jobs = pd.read_csv(local_jobs_cleaned_lemmatized_tokenized_path)
jobs.head()
jobs_sentences = jobs['lemmatized_combined'].tolist()

Training models
---------------




For the following examples, we'll use the Lee Corpus (which you already have if you've installed gensim) for training our model.






In [7]:
#this may be annoying later...
from pprint import pprint

from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

# Set file names for train and test data
#corpus_file = datapath('lee_background.cor')



2020-04-16 14:47:57,046 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 14:47:57,048 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [9]:
model = FT_gensim(size=100)

# build the vocabulary
model.build_vocab(sentences = jobs_sentences)

# train the model
model.train(
    sentences=jobs_sentences, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words
)

print(model)

2020-04-16 14:48:26,090 : INFO : resetting layer weights
2020-04-16 14:48:41,359 : INFO : collecting all words and their counts
2020-04-16 14:48:41,387 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2020-04-16 14:48:41,445 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-04-16 14:48:43,585 : INFO : PROGRESS: at sentence #10000, processed 16817539 words, keeping 31 word types
2020-04-16 14:48:44,301 : INFO : collected 31 word types from a corpus of 22605389 raw words and 13124 sentences
2020-04-16 14:48:44,303 : INFO : Loading a fresh vocabulary
2020-04-16 14:48:44,306 : INFO : effective_min_count=5 retains 31 unique words (100% of original 31, drops 0)
2020-04-16 14:48:44,307 : INFO : effective_min_count=5 leaves 22605389 word corpus (100% of original 22605389, drops 0)
2020-04-16 14:48:44,315 : INFO : deleting the raw counts dictionary of 31 items
2020-04-16 14:48:44,

FastText(vocab=31, size=100, alpha=0.025)


Training hyperparameters
^^^^^^^^^^^^^^^^^^^^^^^^




Hyperparameters for training the model follow the same pattern as Word2Vec. FastText supports the following parameters from the original word2vec:

- model: Training architecture. Allowed values: `cbow`, `skipgram` (Default `cbow`)
- size: Size of embeddings to be learnt (Default 100)
- alpha: Initial learning rate (Default 0.025)
- window: Context window size (Default 5)
- min_count: Ignore words with number of occurrences below this (Default 5)
- loss: Training objective. Allowed values: `ns`, `hs`, `softmax` (Default `ns`)
- sample: Threshold for downsampling higher-frequency words (Default 0.001)
- negative: Number of negative words to sample, for `ns` (Default 5)
- iter: Number of epochs (Default 5)
- sorted_vocab: Sort vocab by descending frequency (Default 1)
- threads: Number of threads to use (Default 12)


In addition, FastText has three additional parameters:

- min_n: min length of char ngrams (Default 3)
- max_n: max length of char ngrams (Default 6)
- bucket: number of buckets used for hashing ngrams (Default 2000000)


Parameters ``min_n`` and ``max_n`` control the lengths of character ngrams that each word is broken down into while training and looking up embeddings. If ``max_n`` is set to 0, or to be lesser than ``min_n``\ , no character ngrams are used, and the model effectively reduces to Word2Vec.



To bound the memory requirements of the model being trained, a hashing function is used that maps ngrams to integers in 1 to K. For hashing these character sequences, the `Fowler-Noll-Vo hashing function <http://www.isthe.com/chongo/tech/comp/fnv>`_ (FNV-1a variant) is employed.




**Note:** As in the case of Word2Vec, you can continue to train your model while using Gensim's native implementation of fastText.




## Now we have a fastText model trained on Resumes
<br>
Now we need to take individual job postings to compare

In [10]:
# load jobs into df

local_resumes_cleaned_lemmatized_tokenized_path = '/Users/richardkuzma/coding/NLP_projects/job_recommender_project/data/cleaned_lemmatized_tokenized_resume_dataset_maitrip.csv'

resumes = pd.read_csv(local_resumes_cleaned_lemmatized_tokenized_path)



In [11]:
resumes.head()

,ID,Category,dirty_resume,resume,tokenized_resume,lemmatized_resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box...",john h smith phr po box callahan fl infog...,"['john', 'h', 'smith', 'phr', 'po', 'box', 'ca...","['john', 'h', 'smith', 'phr', 'po', 'box', 'ca..."
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...,name surname address mobile noemail personal p...,"['name', 'surname', 'address', 'mobile', 'noem...","['name', 'surname', 'address', 'mobile', 'noem..."
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...,anthony brown hr assistant areas expertise per...,"['anthony', 'brown', 'hr', 'assistant', 'areas...","['anthony', 'brown', 'hr', 'assistant', 'area'..."
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...,satheesh email id career objective pursue gro...,"['satheesh', 'email', 'id', 'career', 'objecti...","['satheesh', 'email', 'id', 'career', 'objecti..."
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert...",human resources director expert organizational...,"['human', 'resources', 'director', 'expert', '...","['human', 'resource', 'director', 'expert', 'o..."


In [13]:
resumes_list = resumes['lemmatized_resume'].tolist()

In [18]:
resumes.shape[0]

1219

In [20]:
resumes.iloc[3]['lemmatized_resume']

"['satheesh', 'email', 'id', 'career', 'objective', 'pursue', 'growth', 'oriented', 'career', 'progressive', 'company', 'provides', 'scope', 'apply', 'knowledge', 'skill', 'would', 'help', 'contribute', 'best', 'organization', 'personal', 'skill', 'enjoy', 'strong', 'logical', 'analytical', 'communication', 'interpersonal', 'skill', 'work', 'dedication', 'determination', 'high', 'energy', 'work', 'ethic', 'commitment', 'client', 'service', 'easily', 'adaptable', 'new', 'technology', 'minimum', 'training', 'interested', 'learning', 'new', 'thing', 'implementation', 'ability', 'work', 'individually', 'group', 'work', 'experience', 'total', 'experience', 'year', 'month', 'presently', 'working', 'interra', 'system', 'india', 'pvt', 'ltd', 'executive', 'hrd', 'bangalore', 'october', 'till', 'date', 'manpower', 'planning', 'recruitment', 'xefxxae', 'xefxxae', 'xefxxae', 'xefxxae', 'xefxxae', 'xefxxae', 'xefxxae', 'xefxxae', 'xefxxae', 'xefxxae', 'xefxxae', 'identification', 'various', 'chann

In [32]:
def pick_resume():
    print("There are {} resumes".format(resumes.shape[0]))
    
    # Select a random int from 0 to length of rjob set
    rand_int = np.random.randint(1, resumes.shape[0]+1)
    
    
    
#     selection = 2
    selection = rand_int
    
    
    
    print ('\nselected resume is ID #{}'.format(selection))
    
    # pick the job text and ID associated with the random int
    resume_label = resumes.iloc[selection - 1, resumes.columns.get_loc('ID')] #we could grab ID, but this works for non-indexed labels too
    resume_text = resumes.iloc[selection - 1 ]['resume'][:500] 
    
    
    print('Resume ID is: {}'.format(resume_label))
    print('Resume text is (500 chars): {}'.format(resume_text))
    
    #Convert the sample document into a list and use the infer_vector method to get a vector representation for it
    resume_text_to_process = resumes['lemmatized_resume'][selection - 1]
    
    return resume_text_to_process


In [43]:
def given_resume_find_similar_jobs(resume_you_pick):
  
    #find all distances between chosen job and each resume
    temp_distance = []
    min_dist = float("inf")
    min_index = float("inf")



#     for i in range (0, 10):
    for i in range (0, len(jobs_sentences)):
        dist = model.wmdistance(resume_you_pick, jobs_sentences[i])
        if dist < min_dist:
            min_dist = dist
            min_index = i
        temp_distance.append((dist, i))

    #sort list of tuples
    temp_distance.sort(key = operator.itemgetter(0))

    return temp_distance

    

In [66]:
def print_similar_jobs(list_of_similar_jobs):
    
    num_similar = 10 #or 10, 20, 25
    print('\nPrinting {} most similar jobs for this candidate...\n'.format(num_similar))
    for i in range(0,num_similar):
        print('\n#{} most similar job'.format(i+1))
        print('Job ID from list: {}'.format(list_of_similar_jobs[i][1]))
        print('WM Distance: {}'.format(list_of_similar_jobs[i][0]))
        print('Job ID from df: {}'.format(jobs.iloc[list_of_similar_jobs[i][1]]['label']))
        print('Job title: {}'.format(jobs.iloc[list_of_similar_jobs[i][1]]['Title']))
        print('Company: {}'.format(jobs.iloc[list_of_similar_jobs[i][1]]['Company']))
        print('Job Description: {}'.format(jobs.iloc[list_of_similar_jobs[i][1]]['JobDescription']))


In [65]:
def print_dissimilar_jobs(list_of_similar_jobs):
    
    num_dissimilar = 10 #or 10, 20, 25
    print('\nPrinting {} most similar jobs for this candidate...\n'.format(num_dissimilar))
    for i in range(0,num_dissimilar):
        print('\n#{} least similar job'.format(i+1))
        print('Job ID from list: {}'.format(list_of_similar_jobs[-(1+i)][1]))
        print('WM Distance: {}'.format(list_of_similar_jobs[-(1+i)][0]))
        print('Job ID from df: {}'.format(jobs.iloc[list_of_similar_jobs[-(1+i)][1]]['label']))
        print('Job title: {}'.format(jobs.iloc[list_of_similar_jobs[-(1+i)][1]]['Title']))
        print('Company: {}'.format(jobs.iloc[list_of_similar_jobs[-(1+i)][1]]['Company']))
        print('Job Description: {}'.format(jobs.iloc[list_of_similar_jobs[-(1+i)][1]]['JobDescription']))

In [61]:
chosen_resume = pick_resume()

There are 1219 resumes

selected resume is ID #806
Resume ID is: 806
Resume text is (500 chars): corporate finance analyst resume scott buhl  main street san francisco ca  home    cell    emailexamplecom professional summary professional corporate financial analyst significant experience public private corporate reporting needs adept departmental corporate budgeting utilizing variety financial spreadsheet programs developing methods reporting data executive team specializes analyzing dynamic financial data developing financial reports core qualifications certified public accountant experien


In [62]:
ordered_job_list = given_resume_find_similar_jobs(chosen_resume)

/Users/richardkuzma/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  if sys.path[0] == '':
2020-04-16 17:09:27,094 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:27,104 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 8144 corpus positions)
2020-04-16 17:09:27,138 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:27,146 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3309 corpus positions)
2020-04-16 17:09:27,205 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:27,217 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 6054 corpus positions)
2020-04-16 17:09:27,259 : INFO : adding document #0 to Dictionary(0 uni

2020-04-16 17:09:28,463 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3313 corpus positions)
2020-04-16 17:09:28,501 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:28,504 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3451 corpus positions)
2020-04-16 17:09:28,530 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:28,534 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4403 corpus positions)
2020-04-16 17:09:28,555 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:28,559 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 6493 corpus positions)
2020-04-16 17:09:28,596 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:28,601 : INFO : built Dictionary(31 unique tokens: [' '

2020-04-16 17:09:30,105 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:30,115 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3046 corpus positions)
2020-04-16 17:09:30,171 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:30,176 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3246 corpus positions)
2020-04-16 17:09:30,238 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:30,239 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3503 corpus positions)
2020-04-16 17:09:30,262 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:30,265 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3513 corpus positions)
2020-04-16 17:09:30,293 : INFO : adding document #0 to Dictionary(0 uniq

2020-04-16 17:09:31,420 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3471 corpus positions)
2020-04-16 17:09:31,463 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:31,465 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4133 corpus positions)
2020-04-16 17:09:31,492 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:31,496 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3555 corpus positions)
2020-04-16 17:09:31,543 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:31,545 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4135 corpus positions)
2020-04-16 17:09:31,572 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:31,577 : INFO : built Dictionary(31 unique tokens: [' '

2020-04-16 17:09:32,664 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:32,668 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4061 corpus positions)
2020-04-16 17:09:32,698 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:32,701 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4196 corpus positions)
2020-04-16 17:09:32,738 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:32,740 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3375 corpus positions)
2020-04-16 17:09:32,771 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:09:32,789 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 12156 corpus positions)
2020-04-16 17:09:32,815 : INFO : adding document #0 to Dictionary(0 uni

...

2020-04-16 17:15:46,396 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4389 corpus positions)
2020-04-16 17:15:46,415 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:46,419 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 6018 corpus positions)
2020-04-16 17:15:46,442 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:46,448 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3217 corpus positions)
2020-04-16 17:15:46,469 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:46,473 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4212 corpus positions)
2020-04-16 17:15:46,496 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:46,500 : INFO : built Dictionary(31 unique tokens: [' '

2020-04-16 17:15:47,552 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:47,554 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3916 corpus positions)
2020-04-16 17:15:47,576 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:47,578 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3537 corpus positions)
2020-04-16 17:15:47,598 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:47,600 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3114 corpus positions)
2020-04-16 17:15:47,623 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:47,625 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3451 corpus positions)
2020-04-16 17:15:47,650 : INFO : adding document #0 to Dictionary(0 uniq

2020-04-16 17:15:48,447 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3872 corpus positions)
2020-04-16 17:15:48,470 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:48,472 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4244 corpus positions)
2020-04-16 17:15:48,494 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:48,498 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4732 corpus positions)
2020-04-16 17:15:48,518 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:48,524 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3352 corpus positions)
2020-04-16 17:15:48,544 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:48,550 : INFO : built Dictionary(30 unique tokens: [' '

2020-04-16 17:15:49,383 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:49,387 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4284 corpus positions)
2020-04-16 17:15:49,409 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:49,415 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3848 corpus positions)
2020-04-16 17:15:49,438 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:49,440 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4546 corpus positions)
2020-04-16 17:15:49,467 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:49,469 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3591 corpus positions)
2020-04-16 17:15:49,488 : INFO : adding document #0 to Dictionary(0 uniq

2020-04-16 17:15:50,303 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4075 corpus positions)
2020-04-16 17:15:50,326 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:50,332 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 4917 corpus positions)
2020-04-16 17:15:50,353 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:50,358 : INFO : built Dictionary(31 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3416 corpus positions)
2020-04-16 17:15:50,386 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:50,388 : INFO : built Dictionary(30 unique tokens: [' ', "'", ',', '[', ']']...) from 2 documents (total 3413 corpus positions)
2020-04-16 17:15:50,411 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 17:15:50,413 : INFO : built Dictionary(31 unique tokens: [' '

In [67]:
print_similar_jobs(ordered_job_list)


Printing 10 most similar jobs for this candidate...


#1 most similar job
Job ID from list: 10506
WM Distance: 0.18459808568437922
Job ID from df: 10507
Job title: Chief Accountant
Company: Ecospan CJSC
Job Description: Ecospan CJSC is looking for a qualified candidate to
fulfill the position of the Chief Accountant.

#2 most similar job
Job ID from list: 9446
WM Distance: 0.18971494408090078
Job ID from df: 9447
Job title: Loan Officer
Company: CARD AgroCredit Universal Credit Organization CJSC
Job Description: The incumbent, under the direct supervision of Credit
Manager, will support the development and effective management of credit
portfolio, participate in formulation and upgrading of the lending
strategies and approaches, promote credit and related services of the
company, take appropriate action to reduce the risk of potential losses
through Non Performing Loans (NPLs), as well as provide leadership,
training and support to less experienced loan officers and other staff
member

In [68]:
print_dissimilar_jobs(ordered_job_list)


Printing 10 most similar jobs for this candidate...


#1 least similar job
Job ID from list: 188
WM Distance: 1.2734941712159888
Job ID from df: 189
Job title: Designers
Company: DiOr (Design of Interiors, Offices and Rosariums)
Job Description: To be able to work hard and to make great works, to
make everybody speak about them...!!!

#2 least similar job
Job ID from list: 6077
WM Distance: 0.9903561793619271
Job ID from df: 6078
Job title: PHP Software Developer
Company: SiteMax LLC
Job Description: SiteMax LLC is looking for an experienced PHP Developer
to join its team.

#3 least similar job
Job ID from list: 11087
WM Distance: 0.9817950704358888
Job ID from df: 11088
Job title: English Language Teacher in Kindergarten
Company: Junior Club LLC
Job Description: Junior Club LLC is looking for an English Language
Teacher in Kindergarten.

#4 least similar job
Job ID from list: 10513
WM Distance: 0.9629717412557288
Job ID from df: 10514
Job title: Programmer
Company: VoIPShop Telecommu